# Homework 2

Start by solving **Exercises** 3.6, 3.15 and 3.17.

Let us start by recalling the code for the LU factorization given in the second tutorial.

In [ ]:
function LU(A)
    n, m = size(A) # A is supposed to be a square matrix, so hopefully n and m will be equal.
    
    # We initalize L with zeros and U to be the same as A.
    L = zeros(n,m)
    U = copy(A)
    
    for k in 1:n
        L[k,k] = 1
        for i in (k+1):n
            L[i,k] = U[i,k]/U[k,k]
            U[i,:] = U[i,:] - L[i,k]*U[k,:]
        end
    end
    return L, U
end

## Exercise

Find one matrix $A \in \mathbb R^{3 \times 3}$, which is invertible and has no diagonal element equal to zero, so that the function `LU(A)` fails. You can tell it fails because it will return matrices $L$ and $U$ for which some entries are `NaN` (Not A Number).

In [ ]:
# You have to modify the following matrix
A = [1. 1. 1.
    1. 1. 1.
    1. 1. 1.]
LU(A)

## Exercise

Implement the forward sustitution algorithm to solve $Lx=y$ when $L$ is lower triangular.

In [ ]:
function forward_substitute(L,y)
    n, m = size(L)
    r, = size(y)
    @assert n == m == r
    x = similar(y)

    # Something has to go here
    
    return x
end

## Exercise

Implement the backward sustitution algorithm to solve $Ux=y$ when $U$ is upper triangular.

**Suggestion**. The range `n:-1:1` corresponds to the numbers $\{n, n-1, n-2, \dots, 2, 1\}$ in backward order.

In [ ]:
function backward_substitute(U,y)
    n, m = size(U)
    r, = size(y)
    @assert n == m == r
    x = similar(n)
    
    # Something has to go here!
    
    return x
end

## Exercise

Write a function that solves a linear system $Ax=f$ using Gaussian elimination.

If you want, you can use the `LU` function above and implement backward substitution. Otherwise, you can re-implement Gaussian elimination modifying the right-hand side $f$ along the way.

In [ ]:
function solve(A,f)
    n, m = size(A)
    r, = size(f)
    @assert n == m == r
    
    # Something goes here
    
    return(x)
end

Let us test it.

In [ ]:
A = [1. 2. 3.
4. 5. 6.
0. 9. 15.]
y = [1.
2.
4.]
x = solve(A,y)
A*x - y

Let us try the algorithm with the Hilbert matrix to see how things can go unexpectedly wrong.

Recall the Hilbert matrix which has a very high condition number
$$ H = \begin{pmatrix} 1 & 1/2 & 1/3 & 1/4 & \dots \\ 1/2 & 1/3 & 1/4 & 1/5 & \dots \\ 1/3 & 1/4 & 1/5 & 1/6 & \dots \\ \dots & & & & \dots \end{pmatrix} = \left\{ \frac 1 {i+j-1} \right\}_{ij}.$$

We will try to solve $Hx = y$ for $y = (1,-1,1,-1,1,-1,\dots)$.

The following function takes the dimension **n** as the parameter and attempts to solve the equation $Hx=y$ for $H$ and $y$ as above. After computing $x$, it displays the value of $Hx$. If the computation was exact, $Hx$ should be the same as $y$. But is it?

In [ ]:
# the following function takes the dimension n as the parameter
function solvewithH(n)
    H = [1/(i+j-1) for i in 1:n, j in 1:n]
    y = [(-1)^i for i in 1:n]
    x = solve(H,y)
    println("We got a vector x that multiplied with H gives us")
    display(H*x)
    println("But we wanted to get")
    display(y)
    return
end

In [ ]:
# Let us try a 3x3 system first. It should work fairly well.
solvewithH(3)

In [ ]:
# Let us now try a 15x15 system. It is not so large. It is arguably surprising that we can already see the effects
# of round-off arithmetic.
solvewithH(15)

Let us now construct the $n^2 \times n^2$ matrix corresponding to the elastic membrane problem that we discussed in class.

In [ ]:
function big_matrix(n)
    A = zeros(n^2,n^2)
    for i in 1:n^2
        A[i,i] = 4
        if mod(i,n)!=0
            A[i+1,i] = -1
            A[i,i+1] = -1
        end
        if i<=n^2-n
            A[i,i+n] = -1
            A[i+n,i] = -1
        end
    end
    return A
end
    

Recall that we wanted to make a system $Ax = y$, with the large matrix $A$ defined above, in order to compute the position of all points in an ellastic membrane with a prescripe shape on the boundary. The right-hand side $y$ depends on this boundary value. Let us set it with the following function. It takes a function $f(x,y)$ that determines the vertical displacement of the membrane for any x,y on the boundary of the unit square. We are discretizing the unit square with points from 0 to N+1 on each side.

This is the function that produces the right-hand side. Don't worry if you do not understand how it works.

In [ ]:
function big_rhs(n, f)
    y = zeros(n^2)
    for i in 1:n
        y[i] += f(i/(n+1),0.)
        y[n^2-n+i] += f(i/(n+1),1.)
    end
        
    for i in 1:n
        y[n*i] += f(1.,i/(n+1))
        y[n*i-n+1] += f(0.,i/(n+1))
    end
    return y
end    

So, it is time to try it. If everything goes well, the following code should produce a pretty picture of an elastic membrane as if it was being pulled down on two opposing edges, with the other two edgest fixed to the boundary.

In [ ]:
n = 10
y = big_rhs(n,(x,y)->1 - sin(pi*x))
A = big_matrix(n)
x = solve(A,y)
membrane = reshape(x,n,n)
partition = 1/(n+1) : 1/(n+1) : 1-1/(n+1)
using CairoMakie
surface(partition,partition,membrane)

Try changing the value of $n$ for a larger one. The shape should look smoother and more realistic for large $n$. But the computation will also take longer. Eventually it will be too long and you would not want to wait for it. If we want to solve an equation like this with a very fine mesh, we should rethink how we solve the linear system.